In [1]:
#@title Automatic Setup
%pwd
!git clone https://github.com/jaywalnut310/vits.git
!python --version
%cd vits/


Cloning into 'vits'...


Python 3.11.0
f:\Text to Speech\vits


In [1]:
!pip install Cython
!pip install librosa
!pip install phonemizer
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode

  Obtaining dependency information for Cython from https://files.pythonhosted.org/packages/db/3e/8cf60af86446b129e79adee5d9131ec7eb043297e20c04bc31670331325c/Cython-3.0.2-cp310-cp310-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.8 MB 2.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.8 MB 3.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.7/2.8 MB 5.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.5/2.8 MB 8.8 MB/s eta 0:00:01
   -------------------------------------- - 2.7/2.8 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------  2.8/2.8 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 9.8 MB/s eta 0:00:00
  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.

In [2]:
%pwd
!python --version
%cd vits/

%cd monotonic_align/
%mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ../
%pwd

Python 3.10.0
f:\Text to Speech\vits
f:\Text to Speech\vits\monotonic_align


A subdirectory or file monotonic_align already exists.


Compiling F:\Text to Speech\vits\monotonic_align\core.pyx because it changed.
[1/1] Cythonizing F:\Text to Speech\vits\monotonic_align\core.pyx
running build_ext
building 'monotonic_align.core' extension
creating build\temp.win-amd64-3.10
creating build\temp.win-amd64-3.10\Release
creating build\temp.win-amd64-3.10\Release\Text to Speech
creating build\temp.win-amd64-3.10\Release\Text to Speech\vits
creating build\temp.win-amd64-3.10\Release\Text to Speech\vits\monotonic_align
F:\Visual Studio\VC\Tools\MSVC\14.37.32822\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -If:\Text to Speech\.venv\lib\site-packages\numpy\core\include -If:\Text to Speech\.venv\include -IF:\Python 3.10\include -IF:\Python 3.10\Include -IF:\Visual Studio\VC\Tools\MSVC\14.37.32822\include -IF:\Visual Studio\VC\Auxiliary\VS\include -IC:\Program Files (x86)\Windows Kits\10\include\10.0.22621.0\ucrt -IC:\Program Files (x86)\Windows Kits\10\\include\10.0.22621.0\\um -IC:\Program Files (x86)\Windows Kits\1

f:\Text to Speech\vits\monotonic_align\setup.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.core import setup
f:\Text to Speech\.venv\lib\site-packages\Cython\Compiler\Main.py:384: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: F:\Text to Speech\vits\monotonic_align\core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


'f:\\Text to Speech\\vits'

#### For Downloading Language Model

LANG = "tam" 
import os
import subprocess
import locale
locale.getpreferredencoding = lambda: "UTF-8"

def download(lang, tgt_dir="./"):
  lang_fn, lang_dir = os.path.join(tgt_dir, lang+'.tar.gz'), os.path.join(tgt_dir, lang)
  cmd = ";".join([
        f"wget https://dl.fbaipublicfiles.com/mms/tts/{lang}.tar.gz -O {lang_fn}",
        f"tar zxvf {lang_fn}"
  ])
  print(f"Download model for language: {lang}")
  subprocess.check_output(cmd, shell=True)
  print(f"Model checkpoints in {lang_dir}: {os.listdir(lang_dir)}")
  return lang_dir

ckpt_dir = download(LANG)

In [4]:
!pip install numpy==1.24

In [4]:

from IPython.display import Audio
import os
import re
import glob
import json
import tempfile
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import commons
import utils
import argparse
import subprocess

In [5]:
import data_utils

from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(arg=0, lineno=1042)
          10	LOAD_CONST(arg=2, lineno=1042)
          12	BINARY_SUBSCR(arg=None, lineno=1042)
          14	COMPARE_OP(arg=4, lineno=1042)
          16	LOAD_FAST(arg=0, lineno=1042)
          18	LOAD_CONST(arg=1, lineno=1042)
          20	BINARY_SUBSCR(arg=None, lineno=1042)
          22	LOAD_FAST(arg=0, lineno=1042)
          24	LOAD_CONST(arg=3, lineno=1042)
          26	BINARY_SUBSCR(arg=None, lineno=1042)
          28	COMPARE_OP(arg=5, lineno=1042)
          30	BINARY_AND(arg=None, lineno=1042)
          32	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

In [6]:
from monotonic_align import monotonic_align

In [7]:
from models import SynthesizerTrn
from scipy.io.wavfile import write

In [13]:
LANG = "tam"

ckpt_dir = r"F:\Text to Speech\tamil"

In [8]:
def preprocess_char(text, lang=None):
    """
    Special treatement of characters in certain languages
    """
    print(lang)
    if lang == 'ron':
        text = text.replace("ț", "ţ")
    return text

In [9]:
class TextMapper(object):
    def __init__(self, vocab_file):
        self.symbols = [x.replace("\n", "") for x in open(vocab_file, encoding="utf-8").readlines()]
        self.SPACE_ID = self.symbols.index(" ")
        self._symbol_to_id = {s: i for i, s in enumerate(self.symbols)}
        self._id_to_symbol = {i: s for i, s in enumerate(self.symbols)}

    def text_to_sequence(self, text, cleaner_names):
        '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
        Args:
        text: string to convert to a sequence
        cleaner_names: names of the cleaner functions to run the text through
        Returns:
        List of integers corresponding to the symbols in the text
        '''
        sequence = []
        clean_text = text.strip()
        for symbol in clean_text:
            symbol_id = self._symbol_to_id[symbol]
            sequence += [symbol_id]
        return sequence

    def uromanize(self, text, uroman_pl):
        iso = "xxx"
        with tempfile.NamedTemporaryFile() as tf, \
             tempfile.NamedTemporaryFile() as tf2:
            with open(tf.name, "w") as f:
                f.write("\n".join([text]))
            cmd = f"perl " + uroman_pl
            cmd += f" -l {iso} "
            cmd +=  f" < {tf.name} > {tf2.name}"
            os.system(cmd)
            outtexts = []
            with open(tf2.name) as f:
                for line in f:
                    line =  re.sub(r"\s+", " ", line).strip()
                    outtexts.append(line)
            outtext = outtexts[0]
        return outtext

    def get_text(self, text, hps):
        text_norm = self.text_to_sequence(text, hps.data.text_cleaners)
        if hps.data.add_blank:
            text_norm = commons.intersperse(text_norm, 0)
        text_norm = torch.LongTensor(text_norm)
        return text_norm

    def filter_oov(self, text):
        val_chars = self._symbol_to_id
        txt_filt = "".join(list(filter(lambda x: x in val_chars, text)))
        return txt_filt

In [10]:
def preprocess_text(txt, text_mapper, hps, uroman_dir=None, lang=None):
    txt = preprocess_char(txt, lang=lang)
    is_uroman = hps.data.training_files.split('.')[-1] == 'uroman'
    if is_uroman:
        with tempfile.TemporaryDirectory() as tmp_dir:
            if uroman_dir is None:
                cmd = f"git clone git@github.com:isi-nlp/uroman.git {tmp_dir}"
                print(cmd)
                subprocess.check_output(cmd, shell=True)
                uroman_dir = tmp_dir
            uroman_pl = os.path.join(uroman_dir, "bin", "uroman.pl")
            print(f"uromanize")
            txt = text_mapper.uromanize(txt, uroman_pl)
            print(f"uroman text: {txt}")
    txt = txt.lower()
    txt = text_mapper.filter_oov(txt)
    return txt

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Run inference with {device}")
vocab_file = f"{ckpt_dir}/vocab.txt"
config_file = f"{ckpt_dir}/config.json"
assert os.path.isfile(config_file), f"{config_file} doesn't exist"
hps = utils.get_hparams_from_file(config_file)
text_mapper = TextMapper(vocab_file)
net_g = SynthesizerTrn(
    len(text_mapper.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
net_g.to(device)
_ = net_g.eval()

g_pth = f"{ckpt_dir}/G_100000.pth"
print(f"load {g_pth}")

_ = utils.load_checkpoint(g_pth, net_g, None)

Run inference with cpu
load F:\Text to Speech\tamil/G_100000.pth
INFO:root:Loaded checkpoint 'F:\Text to Speech\tamil/G_100000.pth' (iteration 8334)


In [22]:
Text = """
        புதுடெல்லி: காவிரியில் இருந்து உரிய நீரை திறந்துவிடக் கோரி தமிழக அரசு தாக்கல். செய்துள்ள மனு வரும் செப்டம்பர் 21ம் தேதி விசாரணைக்கு எடுத்துக்கொள்ளப்படும் என்று உச்ச நீதிமன்றம் தெரிவித்துள்ளது.

""" 

In [25]:
#@title Generate audio from text { run: "auto" }

txt = Text
txt = preprocess_text(txt, text_mapper, hps, lang=LANG)
stn_tst = text_mapper.get_text(txt, hps)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    hyp = net_g.infer(
        x_tst, x_tst_lengths, noise_scale=.667,
        noise_scale_w=0.8, length_scale=1.0
    )[0][0,0].cpu().float().numpy()

print(f"Generated audio")
Audio(hyp, rate=hps.data.sampling_rate)

tam
Generated audio
